In [1]:
import math
import datetime
from memoize import *

In [2]:
from mpmath import iv, mpf  # interval arithmetic, multiprecision float
iv.dps = 128 # digits of precision
iv.pretty = True # Make it print interval enclosures nicely
Interval = iv.mpf
Exp = iv.exp

In [3]:
def Time():
    print(str(datetime.datetime.now()))

In [4]:
class ContinuedFractionSystem:
    
    def __init__(self, knots, alphabet):
        self.knots = [ Interval(knot) for knot in knots]
        self.alphabet = [ Interval(x) for x in alphabet ]
        self.gamma = min(self.alphabet)
        
    @memoize
    def ContinuedFraction(self, x, m):
        return 1.0/(x + m)
    
    @memoize
    def ContinuedFractionDerivative(self, x, m):
        return -1.0/((x+m)**2.0)
    
    @memoize
    def BoundaryKnots(self, x):
        a = max([ knot for knot in self.knots if knot <= x ])
        b = min([ knot for knot in self.knots if knot >= x ])
        ai = self.knots.index(a)
        bi = self.knots.index(b)
        return (a,b,ai,bi)
    
    def Interpolate(self, v, x):
        (a,b,ai,bi) = self.BoundaryKnots(x)
        if ai == bi:
            return v[ai]
        return ((b-x)*v[ai] + (x-a)*v[bi])/(b-a)
    
    @memoize
    def LowerBoundError(self, s, x, m):
        # Formula on middle of page 8
        theta = self.ContinuedFraction(x,m)
        (a,b,ai,bi) = self.BoundaryKnots(theta)
        return (b-theta)*(theta-a)*s*(2.0*s + 1.0)*Exp(2.0*s*(b-a)/self.gamma)/(self.gamma**2.0)
    
    @memoize
    def LowerCoefficients(self, s):
        return [ [ (abs(self.ContinuedFractionDerivative(x,m))**s) * (1.0 - self.LowerBoundError(s,x,m)) for m in self.alphabet ] for x in self.knots]

    def Lower(self, v, s):
        coefficients = self.LowerCoefficients(s)
        return [ sum( [coefficients[i][j] * self.Interpolate(v, self.ContinuedFraction(x,m)) \
                       for (j,m) in enumerate(self.alphabet)]) for (i,x) in enumerate(self.knots)]
    
    @memoize
    def UpperCoefficients(self, s):
        return [ [ abs(self.ContinuedFractionDerivative(x,m))**s for m in self.alphabet ] for x in self.knots]
    
    def Upper(self, v, s):
        coefficients = self.UpperCoefficients(s)
        return [ sum( [coefficients[i][j] * self.Interpolate(v, self.ContinuedFraction(x,m)) \
                       for (j,m) in enumerate(self.alphabet)]) for (i,x) in enumerate(self.knots)]

In [5]:
def norm(v):
    return sum([ abs(x) for x in v])

In [6]:
def normalize(v):
    k = norm(v)
    return [ x/k for x in v ]

In [7]:
def IsContractive(x0, F, tol=Interval(1e-16)):
    N = len(x0)
    while 1:
        #print("Iterating with x0 = " + str(x0))
        x = F(x0)
        if all( x[i] < x0[i] for i in range(0,N) ):
            return ("Contractive", x0)
        if all( x[i] > x0[i] for i in range(0,N) ):
            return ("Not Contractive", x0)
        if all( abs(x[i] - x0[i]) < tol for i in range(0,N) ):
            return ("Undetermined", x0)
        x0 = normalize(x)

In [8]:
def HausdorffDimensionBounds(continued_fraction_system):
    Lower = continued_fraction_system.Lower
    Upper = continued_fraction_system.Upper
    N = len(continued_fraction_system.knots)
    def BisectionSearch(F):
        def FIsContractive(s, x0 = None):
            if x0 == None:
                x0 = [Interval(1) for i in range(0, N)]
            return IsContractive(x0, lambda x : F(x, s))
        bottom = Interval(0.0)
        top = Interval(1.0)
        point = None
        while 1:
            s = (top + bottom)/2.0
            (status, point) = FIsContractive(s, point)
            if status == "Contractive":
                top = s
            elif status == "Not Contractive":
                bottom = s
            elif status == "Undetermined":
                break
            print(Interval([bottom.a,top.b]))
        return Interval([bottom.a,top.b])
    lower_interval = BisectionSearch(Lower)
    upper_interval = BisectionSearch(Upper)
    print("Lower interval = " + str(lower_interval))
    print("Upper interval = " + str(upper_interval))
    return Interval([lower_interval.a, upper_interval.b])

In [9]:
N = 10000
knots = [ Interval(i)/Interval(N) for i in range(0,N+1) ]
alphabet = [1,2]
continued_fraction_system = ContinuedFractionSystem(knots,alphabet)

In [10]:
Time()
print(HausdorffDimensionBounds(continued_fraction_system))
Time()

2017-04-04 02:59:41.427294
[0.5, 1.0]
[0.5, 0.75]
[0.5, 0.625]
[0.5, 0.5625]
[0.53125, 0.5625]
[0.53125, 0.546875]
[0.53125, 0.5390625]
[0.53125, 0.53515625]
[0.53125, 0.533203125]
[0.53125, 0.5322265625]
[0.53125, 0.53173828125]
[0.53125, 0.531494140625]
[0.53125, 0.5313720703125]
[0.53125, 0.53131103515625]
[0.53125, 0.531280517578125]
[0.5312652587890625, 0.531280517578125]
[0.53127288818359375, 0.531280517578125]
[0.531276702880859375, 0.531280517578125]
[0.5312786102294921875, 0.531280517578125]
[0.53127956390380859375, 0.531280517578125]
[0.531280040740966796875, 0.531280517578125]
[0.5312802791595458984375, 0.531280517578125]
[0.53128039836883544921875, 0.531280517578125]
[0.531280457973480224609375, 0.531280517578125]
[0.5312804877758026123046875, 0.531280517578125]
[0.53128050267696380615234375, 0.531280517578125]
[0.53128050267696380615234375, 0.531280510127544403076171875]
[0.53128050267696380615234375, 0.5312805064022541046142578125]
[0.53128050453960895538330078125, 0.5312